In [ ]:
import json
from pymongo import MongoClient
from bson.objectid import ObjectId
import numpy as np
import random
import re
import os
import faiss
from sentence_transformers import SentenceTransformer
import sys
sys.path.extend(["../../../../", "../../", "../"])

from utils.helper_functions import clean_title
from vector_stores.helper_functions import generate_law_wording
#from BM25.BM25_retriever import BM25Retriever
#from helper_functions import remove_punctuation, lemmatization#, remove_stopwords
from retriever import SAVIARetriever

%load_ext autoreload
%autoreload 2

In [ ]:
client = MongoClient("mongodb://mongo_db:27017")
db_SAVIA = client["SAVIA"]
db_SAVIA_NLU = client["SAVIA_NLU"]
coll_leggi_regionali = db_SAVIA["leggiRegionali"]
print(sorted(db_SAVIA_NLU.list_collection_names()))

In [ ]:
#docs = list(coll_leggi_regionali.find())

In [ ]:
#fields = ['legge', 'titolo', 'summary', 'keywords']
#[v for k, v in random.choice(docs).items() if k in fields]

### Load dataset

In [ ]:
dataset_folder = "../../../../../datasets/instruction_datasets/instruction_dataset_v3"

with open(os.path.join(dataset_folder, "IB_dataset.json"), "r") as f:
    IB_dataset = json.load(f)

with open(os.path.join(dataset_folder, "general_questions_dataset.json"), "r") as f:
    general_questions_dataset = json.load(f)

with open(os.path.join(dataset_folder, "other_questions_dataset.json"), "r") as f:
    other_questions_dataset = json.load(f)

### Test on IB dataset

In [ ]:
retriever = SAVIARetriever(vector_store_folder = "../../../../../SAVIA_vector_stores")

In [ ]:
ind = random.randint(0, len(IB_dataset) - 1)
#ind = 234
print(ind)
item = IB_dataset[ind]
question = item['question']

#question = "Qual è la legge vigente sulla raccolta dei tartufi, e i suoi atti attuativi emanati nel 2021?"
#question = "Quanti sono gli atti attuativi che riguardano la legge n. 11 del 1983 emanati nel 1983?"

context = item['context']
gt = context[0]['legge'].upper().replace(" , ", ", ")

#question = "Quali sono le leggi regionali che riguardano la promozione dell'agricoltura biologica, e i rispettivi atti attuativi?" 
#question = "Quali atti attuativi sono stati emanati riguardo alla legge n. 20/2017 nel 2024?"

print(question)
print()
retrieved_item = retriever.create_prompt(question)
print("num retrieved items:", len(retrieved_item))

print("GT:  ", gt)
if len(retrieved_item) > 0:
    print("PRED:", retrieved_item[0]['legge'].upper())

In [ ]:
retrieved_item

In [ ]:
print(sorted(db_SAVIA.list_collection_names()))

In [ ]:
coll_atti_giunta = db_SAVIA['attiGiuntaRegionale']

In [ ]:
v = [ObjectId('66377ad506974927553477fd'), ObjectId('663cf0c1f42b67f2890eb147'), ObjectId('663c9eeb37cc70e2c6794b8c'), ObjectId('663cf0a9f42b67f2890eb120'), ObjectId('66376df70697492755346b8d')]

res = list(coll_atti_giunta.find({"_id": {"$in": v}}))

In [ ]:
type(res[0]['anno'])

In [ ]:
data = list(db_SAVIA_NLU['leggiRegionaliHQonLaw'].find())

In [ ]:
for item in data[0:]:
    db_SAVIA_NLU['leggiRegionaliHQonLaw'].update_one({ "_id": item['_id']}, { "$rename": { '_id_legge': '_id_law'}})

In [ ]:
item

In [ ]:
for doc in db_SAVIA_NLU['leggiRegionaliHQonLaw'].find()[45:46]:
    print(doc)

### Test on general questions

In [ ]:
retriever = SAVIARetriever(vector_store_folder = "../../../../../SAVIA_vector_stores")

In [ ]:
ind = random.randint(0, len(general_questions_dataset) - 1)
#ind = 3164
print(ind)
question = random.choice(general_questions_dataset[ind]['questions'])

print(question)
retrieved_item = retriever.create_prompt(question)
print("num retrieved items", len(retrieved_item))

In [ ]:
#retrieved_item

### Test on other questions

In [ ]:
retriever = SAVIARetriever(vector_store_folder = "../../../../../SAVIA_vector_stores")

In [ ]:
ind = random.randint(0, len(other_questions_dataset) - 1)
#ind = 3
ind = 25
print(ind)
question = random.choice(other_questions_dataset[ind]['questions'])
#question = "Per accedere ai contributi regionali previsti dalla legge sulla partecipazione (legge 15/2018) ci sono requisiti? Se sì, quali?"
print(question)
retrieved_item = retriever.create_prompt(question)
#print(retrieved_item[0]['legge'], " --- ", retrieved_item[0]['stato'], retrieved_item[0]['_id'])
#retrieved_item

In [ ]:
retrieved_item

In [ ]:
#res = coll_leggi_regionali.find_one({"_id": ObjectId('663f2cb8f8822e25d433e0e1')})

In [ ]:
res

In [ ]:
question = "Per accedere ai contributi regionali previsti dalla legge sulla partecipazione deliberativa ci sono requisiti? Se sì, quali?"

### Test on synthetic questions

In [ ]:
retriever = SAVIARetriever(vector_store_folder = "../../../../../SAVIA_vector_stores")

In [ ]:
#coll_leggiRegionaliHQonArticles = db_SAVIA_NLU['leggiRegionaliHQonArticles']
#leggiRegionaliHQonArticles = list(coll_leggiRegionaliHQonArticles.find())
#_ids_laws = list(set([x['_id_law'] for x in leggiRegionaliHQonArticles]))

#seleziona solo le domande per cui ci sono anche domande sugli articoli nel db SAVIA_NLU 
coll_leggiRegionaliHQonLaw = db_SAVIA_NLU['leggiRegionaliHQonLaw']
leggiRegionaliHQonLaw = list(coll_leggiRegionaliHQonLaw.find())
#leggiRegionaliHQonLaw = [x for x in list(coll_leggiRegionaliHQonLaw.find()) if x['_id_law'] in _ids_laws]
#len(leggiRegionaliHQonLaw)

In [ ]:
ind = random.randint(0, len(leggiRegionaliHQonLaw) - 1)
#ind = 9284
print(ind)
question = leggiRegionaliHQonLaw[ind]['chunk'].strip()
gt = coll_leggi_regionali.find_one({"_id": leggiRegionaliHQonLaw[ind]['_id_law']})['legge']

print(question)
retrieved_item = retriever.create_prompt(question)
print("GT:", gt)
#print("PRED:", retrieved_item[0]['legge'])#, " --- ", retrieved_item[0]['stato'])#, retrieved_item[0]['_id'])
#if gt == retrieved_item[0]['legge']:
#    print("OK")

#retrieved_item

In [ ]:
retrieved_item

In [ ]:
out = coll_leggi_regionali.find_one({"_id": ObjectId('663f1c7ae9c699e78e2b8a74')})

In [ ]:
out['articoli']['Art. 21']

In [ ]:
for ind, doc in enumerate(coll_leggi_regionali.find()):
    if doc['legge'] == "LEGGE REGIONALE 22 ottobre 2018, n. 15":
        print(ind)

### Test on generated questions

In [ ]:
retriever = SAVIARetriever(vector_store_folder = "../../../../../SAVIA_vector_stores")

In [ ]:
index_name = "leggiRegionaliQuestions"
coll_leggi_regionali_questions = db_SAVIA_NLU[index_name]

vector_store_folder = "../../../../../SAVIA_vector_stores"

leggi_regionali_questions_index = faiss.read_index(os.path.join(vector_store_folder , index_name + ".faiss"))
leggi_regionali_questions = list(coll_leggi_regionali_questions.find())
print(len(leggi_regionali_questions))

In [ ]:
ind = random.randint(0, len(leggi_regionali_questions) - 1)
#ind = 1358
print(ind)
leggi_regionali_question = leggi_regionali_questions[ind]
#leggi_regionali_question = leggi_regionali_questions[0]

question = leggi_regionali_question['chunk'].strip().lower()

legge = coll_leggi_regionali.find_one({"_id": leggi_regionali_question['_id_legge']})

print(leggi_regionali_question)
print()
print("Legge:", legge['legge'])
print("Titolo:", legge['titolo'])

retrieved_item = retriever.create_prompt(question)
#print(len(retrieved_item))
retrieved_item


### Test on custom questions

In [ ]:
retriever = SAVIARetriever(vector_store_folder = "../../../../../SAVIA_vector_stores")

In [ ]:
question = "Che legge definisce le norme per la pubblicazione degli atti amministrativi nel bollettino ufficiale regionale?"
retrieved_item = retriever.create_prompt(question)
#print(len(retrieved_item))
retrieved_item

In [ ]:
retrieved_item

In [ ]:
leggi_regionali_questions[0]

In [ ]:
len(retrieved_item)

In [ ]:
related_d

In [ ]:
sorted_item_ids = 

In [ ]:
retrieved_item = retriever.create_prompt(question)
print(len(retrieved_item))
retrieved_item

In [ ]:
retrieved_item

In [ ]:
se sono 2 uguali: setta anno
se è solo 1: guarda se l'anno è diverso dall'anno della legge

In [ ]:
## Massimo 20 atti: in ordine cronologico discendente prima di 

In [ ]:
item = 'LEGGE REGIONALE 04 marzo 2019, n. 2'

question_embeddings = embedding_model.encode([item.lower()])
faiss.normalize_L2(question_embeddings)

similarity, item_ids = index.search(question_embeddings, 1)
similarity = similarity[0][0]
item_ids = item_ids[0][0]
print(similarity)

In [ ]:
chunk = 'quali analisi, relazioni e schede tecniche sono state sviluppate riguardo alla "legge regionale 04 marzo 2019, n. 2" sul tema dell\'esercizio dell\'apicoltura?'
index = faiss.IndexIDMap2(faiss.IndexFlatIP(embedding_dim))
embedding = embedding_model.encode([chunk.lower()])
faiss.normalize_L2(embedding)
index.add_with_ids(embedding, np.array([0]))

In [ ]:
print("num chunks:", index.ntotal)

In [ ]:
len(chunks_list)

In [ ]:
#index = faiss.IndexIDMap2(faiss.IndexFlatIP(embedding_dim))
#embedding = embedding_model.encode([chunk.lower()])
#faiss.normalize_L2(embedding)
#index.add_with_ids(embedding, np.array([0]))

### Load FAISS indices

In [ ]:
vector_store_folder = "../../../../SAVIA_vector_stores"
index_name = "leggiRegionaliKeywords"
coll_leggi_regionali_keywords = db[index_name]

In [ ]:
leggi_regionali_keywords_index = faiss.read_index(os.path.join(vector_store_folder, "leggiRegionaliKeywords.faiss"))

### Merging two indices

In [ ]:
# Assuming db1 and db2 are your two FAISS vectorstores
#index1 = db1.faiss_index
#index2 = db2.faiss_index

# Create a new FAISS index with the same dimension as the original indexes
#new_index = FAISS.create_faiss_index(index1.d)

# Add vectors from both indexes to the new index
#new_index.add(np.vstack((index1.reconstruct(i) for i in range(index1.ntotal))))
#new_index.add(np.vstack((index2.reconstruct(i) for i in range(index2.ntotal))))

# Create a new FAISS vectorstore with the merged index
#new_db = FAISS(new_index, db1.embeddings)

### Retrieve content with BM25

In [ ]:
list_all_laws = []
corpus = []

for ind, doc in enumerate(coll_leggi_regionali.find()[0:]):
    titolo_orig = doc['titolo']
#    titolo = titolo_orig
    titolo = clean_title(titolo_orig).lower()
    out_dict = {"titolo": titolo, "legge": doc["legge"], 
                          "testo": doc['testo'].replace("\n", " ").lower(),#, "numero": doc["numero"],
                        "legge": doc['legge']}#, "numeroAnno": doc["numeroAnno"]}

    if 'articoli' in doc.keys():
        articoli = list(doc['articoli'].values())
        articoli_str = " ".join([x for x in articoli[0:2]]).lower()
        
        out_dict["articoli"] = articoli_str

    list_all_laws.append(out_dict)

    corpus.append(" ".join(list(out_dict.values())))

In [ ]:
bm25_retriever = BM25Retriever(corpus, top_n = 30, threshold = None)

In [ ]:
ind = random.randint(0, len(IB_dataset_train) - 1)
#ind = 825
print(ind)
question = IB_dataset_train[ind]['question']
context = IB_dataset_train[ind]['context']
gt = context[0]['legge'].upper().replace(" , ", ", ")

print(question)

In [ ]:
top_n_inds, top_n_sim = bm25_retriever.retrieve(question)

for ind_sim, ind in enumerate(top_n_inds):
    pred = list_all_laws[ind]['legge'].upper().replace(" , ", ", ")
#    print(str(ind_sim) + " - score " + str(top_n_sim[ind_sim])) 

#    print("PRED: {}".format(pred), "  ----   GT: {}".format(gt))
#    print("titolo:", list_all_laws[ind]['titolo'].upper())

    if pred == gt:
        print("True document")
        print(str(ind_sim) + " - score " + str(top_n_sim[ind_sim])) 

        print("PRED: {}".format(pred), "  ----   GT: {}".format(gt))
        print("titolo:", list_all_laws[ind]['titolo'].upper())
#    print("--------------")

In [ ]:
#top_m = 5
#question_embedding = embedding_model.encode([question.lower()])
#faiss.normalize_L2(question_embedding)

#similarity, item_ids = leggi_regionali_keywords_index.search(question_embedding, top_m)
#similarity = similarity[0]
#item_ids = item_ids[0]

#for ind_item, item_id in enumerate(item_ids[0:]):
#    print(item_id)
#    print("PRED ANSWER:", list_texts[item_id], " - sim:", round(similarity[ind_item], 3))

#    res_chunk = coll_leggi_regionali_keywords.find_one({"_id_faiss": int(item_id)})
#    _id_legge = res_chunk['_id_legge']
#    res_legge = coll_leggi_regionali.find_one({"_id": _id_legge})
#    legge = res_legge['legge']
#    print("legge:", legge, "- sim:", round(similarity[ind_item], 3))

In [ ]:
retrieved_item

In [ ]:
chunk = 'i atti della legge regionale n'

question_embeddings = embedding_model.encode([chunk.lower()])
faiss.normalize_L2(question_embeddings)

In [ ]:
similarity, item_ids = retriever.leggi_regionali_allegati_index.search(question_embeddings, 1)
#similarity = similarity[0]
#item_ids = item_ids[0]
print(similarity, item_ids)

In [ ]:
question_embeddings.shape

In [ ]:
index = faiss.IndexIDMap2(faiss.IndexFlatIP(embedding_dim))
embedding = embedding_model.encode([chunk.lower()])
faiss.normalize_L2(embedding)
index.add_with_ids(embedding, np.array([0]))

In [ ]:
question_embeddings = embedding_model.encode(["provvedimenti attuativi".lower()])
faiss.normalize_L2(question_embeddings)

similarity, item_ids = index.search(question_embeddings, 1)
similarity = similarity[0][0]
item_ids = item_ids[0][0]
print(similarity)

In [ ]:
question

In [ ]:
good = 0
bad = 0

for ind in range(0, len(IB_dataset_train)):
#for ind in range(0, 1000):
    question = IB_dataset_train[ind]['question']
    context = IB_dataset_train[ind]['context']
    gt = context[0]['legge'].upper().replace(" , ", ", ")

    retrieved_item = retriever.retrieve(question)
    #print(retrieved_item)
    
    if retrieved_item.upper() != gt:
#        print("wrong", retrieved_item, "  -----   ", gt)
        bad += 1
    else:
#        print("ok")
        good += 1

print("accuracy:", good/(good+bad))

In [ ]:
#results

In [ ]:
retrieved_item = retriever.retrieve(question)
retrieved_item

In [ ]:
chunk = "RETE ESCURSIONISTICA DELL'EMILIA-ROMAGNA E VALORIZZAZIONE DELLE ATTIVITÀ ESCURSIONISTICHE"

In [ ]:
index = faiss.IndexIDMap2(faiss.IndexFlatIP(embedding_dim))
embedding = embedding_model.encode([chunk.lower()])
faiss.normalize_L2(embedding)
index.add_with_ids(embedding, np.array([0]))

In [ ]:
question_embeddings = embedding_model.encode([question.lower()])
faiss.normalize_L2(question_embeddings)

similarity, item_ids = index.search(question_embeddings, 1)
similarity = similarity[0][0]
item_ids = item_ids[0][0]
print(similarity)

In [ ]:
res = coll_leggi_regionali.find_one({"_id": retriever._id_leggi[579]})
res['legge']

In [ ]:
list(res['testo_orig']['articoli'].values())[0:2]

In [ ]:
retriever._id_leggi[579]

In [ ]:
res = coll_leggi_regionali.find_one({"_id": retriever._id_leggi[579]})
res['legge']

In [ ]:
top_m = 1
question_embedding = embedding_model.encode(texts)
faiss.normalize_L2(question_embedding)

similarity, item_ids = leggi_regionali_keywords_index.search(question_embedding, top_m)
similarity = similarity[:, 0]
item_ids = item_ids[:, 0]

max_similarity_id = np.argmax(similarity)
best_similarity = similarity[max_similarity_id]
best_item_id = item_ids[max_similarity_id]


    res_chunk = coll_leggi_regionali_keywords.find_one({"_id_faiss": int(item_id)})
    _id_legge = res_chunk['_id_legge']
    res_legge = coll_leggi_regionali.find_one({"_id": _id_legge})
    legge = res_legge['legge']
    print("legge:", legge, "- sim:", round(best_similarity[ind_item], 3))

for ind_item, item_id in enumerate(best_item_id[0:]):
#    print(item_id)
#    print("PRED ANSWER:", list_texts[item_id], " - sim:", round(similarity[ind_item], 3))

    res_chunk = coll_leggi_regionali_keywords.find_one({"_id_faiss": int(item_id)})
    _id_legge = res_chunk['_id_legge']
    res_legge = coll_leggi_regionali.find_one({"_id": _id_legge})
    legge = res_legge['legge']
    print("legge:", legge, "- sim:", round(best_similarity[ind_item], 3))

In [ ]:
best_item_id

In [ ]:
item_ids

In [ ]:
np.argmax(similarity)

### Test search over a subset

In [ ]:
import numpy as np
import faiss

# Set random seed for reproducibility
np.random.seed(0)

# Create a set of 5 small binary vectors
vectors = np.array([[1., 0., 1.],
                    [0., 1., 0.],
                    [1., 1., 0.],
                    [0., 0., 1.],
                    [1., 0., 0.]]).astype(np.float32)

# Initialize an index with the Hamming distance measure
index = faiss.IndexFlatL2(vectors.shape[1])

# Add vectors to the index
index.add(vectors)

# Perform a similarity search
query_vector = np.array([[1, 1, 0]], dtype=np.float32)
k = 3  # Number of nearest neighbors to retrieve

distances, indices = index.search(query_vector, k)
print(indices)

In [ ]:
res = np.int64(4)

In [ ]:
type(res)

In [ ]:
filter_ids = np.array([0, 1, 2, 3], dtype = np.int64)
id_selector = faiss.IDSelectorArray(4, filter_ids)

#filtered_distances, filtered_indices = index.search(query_vector, k, params=faiss.SearchParametersIVF(sel=id_selector))
#print(filtered_indices)

In [ ]:
help(faiss.IDSelectorArray)

In [ ]:
type(vectors[0][0])

In [ ]:
coll_leggi_regionali_keywords.find_one({"_id_faiss": int(item_id)})

In [ ]:
ddfor doc in coll_leggi_regionali_keywords.find()[240998:240999]:
    print(doc)

In [ ]:
d

In [ ]:
top_n_inds, top_n_sim = retriever.retrieve(question)
d
for ind_sim, ind in enumerate(top_n_inds):dddd
    print(str(ind_sim) + " - score " + stdddr(toddp_n_sim[ind_sim])) 
    print("titolo:", list_all_laws[ind]['titolo'].upper())
    print("legge: {}".format(list_all_laws[ind]['legge'].upper()))
          
    print("--------------")